In [1]:
import unicodedata
import string
import re
import random

import torch
import torch.cuda as cuda
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

# Getting the data

In [2]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 20

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2 # Count SOS and EOS
      
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
#     s = unicodeToAscii(s.lower().strip())
#     s = re.sub(r"([.\.!?])", r"\1", s)
    s = s.lower().strip()
    s = re.sub(r"(')", r"", s)
    s = re.sub(r"[^a-zA-Z]+", r" ", s).strip()
    return s

In [4]:
def prepare_data(fname):
    # fname refers to the question answer file which is tab seperated
    
    print("Reading lines...")

    # Read the file and split into lines
    lines = open(fname).read().strip().split('\n')
    
    lines = [normalizeString(line) for line in lines]
    
    questions = lines[0::2]
    answers = lines[1::2]
    
    pairs = [(q, a) for (q, a) in zip(questions, answers)]
    # Reverse pairs, make Lang instances
    lang = Lang('eng')
    for line in lines:
        lang.addSentence(line)
    print "Done"
#     pairs = (questions, answers)
    return lang, pairs

In [5]:
lang, pairs = prepare_data('../res/all_dialogs.txt')

Reading lines...
Done


# Model

In [6]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, emb_dims):
        super(EncoderRNN, self).__init__()
#         self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.emb_dims = emb_dims
        
        self.embedding = nn.Embedding(lang.n_words, emb_dims).cuda()
        self.rnn = nn.LSTM(emb_dims, hidden_size).cuda()
        
    def forward(self, input, hidden):
        embedded = self.embedding(input)
        embedded = embedded.view(1, 1, -1)
        output = embedded
        output, hidden = self.rnn(output, hidden)
        return output, hidden

    def initHidden(self):
        return (Variable(cuda.FloatTensor(1, 1, self.hidden_size).zero_()),
               Variable(cuda.FloatTensor(1, 1, self.hidden_size).zero_()))

In [7]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, context_size, output_size, emb_dims):
        super(DecoderRNN, self).__init__()
#         self.n_layers = n_layers
        self.emb_dims = emb_dims
        self.hidden_size = hidden_size
        self.context_size = context_size
        self.embedding = nn.Embedding(lang.n_words, emb_dims).cuda()
        self.rnn = nn.LSTM(emb_dims + context_size, hidden_size).cuda()
        self.out = nn.Linear(hidden_size, output_size).cuda()
        self.softmax = nn.LogSoftmax()
        
    def forward(self, input, hidden, context):
        output = self.embedding(input).view(1, -1)
        items = [output, context.view(1, -1)]
        output = torch.cat(items, 1).view(1, 1, -1)
        output = F.relu(output)
#         print output.size()
        output, hidden = self.rnn(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return (Variable(cuda.FloatTensor(1, 1, self.hidden_size).zero_()),
               Variable(cuda.FloatTensor(1, 1, self.hidden_size).zero_()))

# Test models

In [8]:
import torch.cuda as cuda
import torch.autograd as ag
question = ['how']
answer = ['i']
inputs = ag.Variable( cuda.LongTensor([ lang.word2index[t] for t in question]))
outputs = ag.Variable( cuda.LongTensor([ lang.word2index[t] for t in answer]))
input_size = 4
hidden_size = 10
output_size = 4
emb_dims = 100
encoder = EncoderRNN(input_size, hidden_size, emb_dims)
decoder = DecoderRNN(hidden_size, hidden_size, output_size, emb_dims)
criterion = nn.NLLLoss()

In [9]:
henc = encoder.initHidden()
out, hstate = encoder.forward(inputs[0], henc)
dec = decoder.initHidden()
dec_output, hstate = decoder.forward(outputs[0], dec, out)

# Prepare for training

In [10]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def variableFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return Variable(cuda.LongTensor(indexes).view(-1, 1))

def variablesFromPair(pair):
    input_variable = variableFromSentence(lang, pair[0])
    target_variable = variableFromSentence(lang, pair[1])
    return (input_variable, target_variable)

In [11]:
teacher_forcing_ratio = 0.5

def train(input_variable, target_variable, encoder, decoder, encoder_optimizer, 
          decoder_optimizer, criterion, max_length=MAX_LENGTH+10):
    
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_variable.size()[0]
    target_length = target_variable.size()[0]
#     encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = Variable(cuda.FloatTensor(max_length, encoder.hidden_size).zero_())
    loss = 0

    for ei in range(min(input_length, max_length)):
        encoder_output, encoder_hidden = encoder(input_variable[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0][0]

    decoder_input = Variable(cuda.LongTensor([[SOS_token]]))
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_output)
            loss += criterion(decoder_output[0], target_variable[di])
            decoder_input = target_variable[di] # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(min(target_length, max_length)):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_output)
            topv, topi = decoder_output.data.topk(1)
            ni = topi[0][0]
            decoder_input = Variable(cuda.LongTensor([[ni]]))
            loss += criterion(decoder_output[0], target_variable[di])
            if ni == EOS_token:
                break

    loss.backward()
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.data[0] / target_length

In [12]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [13]:
def trainEpochs(encoder, decoder, n_epochs, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0 # Reset every print_every
    plot_loss_total = 0 # Reset every plot_every
    
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [variablesFromPair(random.choice(pairs)) for i in range(n_epochs)]
    print "Starting training"
    criterion = nn.NLLLoss()
    
    for epoch in range(1, n_epochs + 1):
        training_pair = training_pairs[epoch - 1]
        input_variable = training_pair[0]
        target_variable = training_pair[1]
        loss = train(input_variable, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss
        if epoch % print_every == 0:
            print "Loss:", loss
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
#             print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs), epoch, epoch / n_epochs * 100, print_loss_avg))
        
        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(loss)
            plot_loss_total = 0
    
    showPlot(plot_losses)

In [14]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
%matplotlib inline

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2) # this locator puts ticks at regular intervals
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [15]:
hidden_size = 256
emb_dims = 200
encoder = EncoderRNN(lang.n_words, hidden_size, emb_dims)
decoder = DecoderRNN(hidden_size, hidden_size, lang.n_words, emb_dims)
trainEpochs(encoder, decoder, 7500, print_every=50)

Starting training
Loss: 10.3386408488
Loss: 0.730105400085
Loss: 0.895856163718
Loss: 8.44246843126
Loss: 8.39349555969
Loss: 4.80534210205
Loss: 2.54293842316
Loss: 10.647388659
Loss: 5.89068937302
Loss: 8.04115549723
Loss: 7.57416742498
Loss: 5.55846405029
Loss: 7.39270324707
Loss: 1.10300276015
Loss: 2.34219474792
Loss: 0.899946576073
Loss: 6.65376586914
Loss: 3.19208641052
Loss: 7.64336355109
Loss: 5.8525967598
Loss: 7.54841450283


KeyboardInterrupt: 